In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import torch
import socialforce

_ = torch.manual_seed(42)

# Diamond

## Parametric

This is an extension of the {ref}`pedped-2d` example to study the robustness
of the inference process to potentials with gradients that change orientation.
We use a modified $V(b)$ potential that could be described as a "diamond"
of height $V_0$ and with a half-width of $\sigma$:
\begin{align}
    V(b) &= V_0 \max\left(0, 1 - \frac{|x_1| + |x_2|}{2\sigma} \right)
\end{align}
with its two parameters $V_0$ and $\sigma$.

In [ ]:
V = socialforce.potentials.PedPedPotentialDiamond(sigma=0.5)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

In [ ]:
V_asym_sd = socialforce.potentials.PedPedPotentialDiamond(sigma=0.5, asymmetry_angle=-20.0, speed_dependent=True)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V_asym_sd, ax1)
    socialforce.show.potential2D_grad(V_asym_sd, ax2)

## Scenarios

Now we use the above parametric potential in a simulation to generate synthetic
scenarios according to this potential.
We generate {ref}`Circle and ParallelOvertake scenarios <scenarios>`.

In [ ]:
circle = socialforce.scenarios.Circle(ped_ped=V)
parallel = socialforce.scenarios.ParallelOvertake(ped_ped=V)
scenarios = circle.generate(20) + parallel.generate(20)
true_experience = socialforce.Trainer.scenes_to_experience(scenarios, radius=3.0)

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas() as ax:
    socialforce.show.states(ax, scenarios[-1], zorder=10)
    for scene in scenarios[:-1]:
        socialforce.show.states(ax, scene, alpha=0.1)

## MLP Inference

We construct an coordinate-based MLP with Fourier 
Features {cite}`tancik2020fourier,rahimi2007random`.

In [ ]:
V = socialforce.potentials.PedPedPotentialMLP2D(hidden_units=128)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

In [ ]:
simulator = socialforce.Simulator(ped_ped=V)
opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator, opt).loop(3, true_experience)

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

## Fourier Features

We construct an coordinate-based MLP with Fourier 
Features {cite}`tancik2020fourier,rahimi2007random`.
This coordinate-based MLP with the Fourier Feature operator FF is:
\begin{align}
    \textrm{MLP}(b) &= \textrm{Softplus} \;\; L_{1\times128} \;\; \textrm{Softplus} \;\; L_{128\times256} \;\; \textrm{FF}_{256 \times 3} \;\; (b, d_{\perp}, d_{\parallel})
\end{align}


In [ ]:
V = socialforce.potentials.PedPedPotentialMLP2D(hidden_units=128, n_fourier_features=256)
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

In [ ]:
# HIDE OUTPUT
simulator = socialforce.Simulator(ped_ped=V)
opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator, opt).loop(10, true_experience)

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)